In [ ]:
!pip install -q groq "transformers==4.51.3" "datasets==3.5.0" "huggingface-hub==0.30.2" "peft==0.15.2"

In [ ]:
import os
from groq import Groq

In [ ]:
from transformers import AutoTokenizer

base_model_id = "Qwen/Qwen3-32B"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

In [ ]:
from google.colab import userdata
groq_api_key = userdata.get('GROQ_API_KEY')

In [ ]:
client = Groq(
    api_key=groq_api_key,
)

## Setting up prompts to play Wordle

In [ ]:
SYSTEM_PROMPT = """
You are playing Wordle, a word-guessing game.

### Game Rules:
- You have **6 tries** to guess a secret **5-letter** word.
- Each guess must be a valid **5-letter English word**.
- After each guess, you will receive feedback indicating how close
your guess was.

### Feedback Format:
Each letter in your guess will receive one of three symbols:
1. ✓ : The letter is in the word and in the CORRECT position.
2. - : The letter is in the word but in the WRONG position.
3. x : The letter is NOT in the word.

### Example:
Secret Word: BRISK

Guess 1: STORM → Feedback: S(-) T(x) O(x) R(-) M(x)
Guess 2: BRAVE → Feedback: B(✓) R(✓) A(x) V(x) E(x)
Guess 3: BRISK → Feedback: B(✓) R(✓) I(✓) S(✓) K(✓)

### Response Format:
Think through the problem and feedback step by step. Make sure to
first add your step by step thought process within <think> </think>
tags. Then, return your guessed word in the following format:
<guess> guessed-word </guess>.
"""

In [ ]:
from dataclasses import dataclass
from enum import Enum
from typing import List


class LetterFeedback(Enum):
    CORRECT = "✓"
    WRONG_POS = "-"
    WRONG_LETTER = "x"


@dataclass
class GuessWithFeedback:
    guess: str
    feedback: List[LetterFeedback]

    def __repr__(self) -> str:
        """Returns a readable string showing the guess alongside
        its letter-by-letter feedback."""
        feedback_str = " ".join(f"{letter}({fb.value})" for letter, fb in zip(self.guess, self.feedback))
        return f"{self.guess} → Feedback: {feedback_str}"

In [ ]:
def render_user_prompt(past_guesses: List[GuessWithFeedback]) -> str:
    """Creates a user-facing prompt that includes past guesses
    and their feedback."""
    prompt = "Make a new 5-letter word guess."
    if past_guesses:
        prompt += "\n\nHere is some previous feedback:"
        for i, guess in enumerate(past_guesses):
            prompt += f"\nGuess {i+1}: {guess}"
    return prompt

## Play a single turn of Wordle
Start by setting up the prompt with feedback for two prior guesses:

In [ ]:
secret_word = "CRAFT"

past_guesses = [
    GuessWithFeedback(
        "CRANE", [
            LetterFeedback.CORRECT,
            LetterFeedback.CORRECT,
            LetterFeedback.CORRECT,
            LetterFeedback.WRONG_LETTER,
            LetterFeedback.WRONG_LETTER,
        ]),
    GuessWithFeedback(
        "CRASH", [
            LetterFeedback.CORRECT,
            LetterFeedback.CORRECT,
            LetterFeedback.CORRECT,
            LetterFeedback.WRONG_LETTER,
            LetterFeedback.WRONG_LETTER,
        ]),
]

In [ ]:
messages = [
        {
            "role": "system",
            "content": SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": render_user_prompt(past_guesses)
        },
        {
            "role": "assistant",
            "content": "Let me solve this step by step.\n<think>"
        }
    ]

In [ ]:
def generate_stream(prompt: str, adapter_id: str = "") -> str:
    """Streams a model-generated response from a prompt in
    real-time and prints it as it arrives."""
    response = client.chat.completions.create(
        model=adapter_id,
        messages =prompt,
        # Produce deterministic responses for evaluation
        max_tokens=2048,
        stream=True,
    )

    completion = ""
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
            content = chunk.choices[0].delta.content
            print(content, end="", flush=True)
            completion += content
    print()

    return completion

In [ ]:
base_completion = generate_stream(prompt = messages, adapter_id = base_model_id)

<think>
Okay, let's try to figure out the next guess based on the feedback given. 

First, the previous guesses and their feedback are:
1. CRANE → C(✓), R(✓), A(✓), N(x), E(x)
2. CRASH → C(✓), R(✓), A(✓), S(x), H(x)

So the secret word must have C, R, and A in the first three positions. Because in both guesses, the first three letters (C, R, A) are marked as correct with ✓, which means they are in the right positions.

The fourth letter in the first guess was N (x), meaning N is not in the word. In the second guess, the fourth letter is S (x), so S is also not present. Similarly, the fifth letter in the first guess was E (x), and in the second guess it's H (x), so E and H are excluded.

So the correct word starts with C-R-A, and we need to determine the fourth and fifth letters. Since none of the other letters (N, S, E, H) from previous guesses are in the word, we need to find a valid 5-letter word that starts with CRA and has the fourth and fifth letters not in the excluded ones. Also

## Playing a game of wordle

Start by setting up a helper function that gets feedback on a guess:

In [ ]:
import re

def get_feedback(guess: str, secret_word: str) -> List[LetterFeedback]:
    valid_letters = set(secret_word)
    feedback = []
    for letter, secret_letter in zip(guess, secret_word):
        if letter == secret_letter:
            feedback.append(LetterFeedback.CORRECT)
        elif letter in valid_letters:
            feedback.append(LetterFeedback.WRONG_POS)
        else:
            feedback.append(LetterFeedback.WRONG_LETTER)
    return feedback

Now create a `next_turn` function that incorporates feedback on a guess into the prompt to the LLM:

In [ ]:
secret_word = "BRICK"
past_guesses = []
adapter_id = "Qwen/Qwen3-32B"

In [ ]:
messages = [
        {
            "role": "system",
            "content": SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": render_user_prompt(past_guesses)
        },
        {
            "role": "assistant",
            "content": "Let me solve this step by step.\n<think>"
        }
    ]

In [ ]:
def next_turn(
    past_guesses: List[GuessWithFeedback],
    secret_word: str,
    adapter_id = ""
):
    completion = generate_stream(messages, adapter_id)
    match = re.search(
        r"<guess>\s*(.*?)\s*</guess>", completion, re.DOTALL
    )
    if not match:
        raise RuntimeError("invalid guess")

    guess = match.group(1).upper()
    feedback = get_feedback(guess, secret_word)
    past_guesses.append(GuessWithFeedback(guess, feedback))
    print("\n\n")
    print(("-" * 100) + "\n")
    for past_guess in past_guesses:
        print(past_guess)

    if guess == secret_word:
        print("🎉 SUCCESS 🎉")
    elif len(past_guesses) >= 6:
        print("❌ better luck next time... ❌")

In [ ]:
next_turn(past_guesses, secret_word, adapter_id)

<think>
Okay, let's see. I need to figure out the secret 5-letter Wordle word. Since I have no prior guesses yet, my first step is to make an initial guess that can give me the most information. A common strategy is to start with a word that has a mix of vowels and consonants because that can help narrow down possibilities quickly. Words like "CRANE" or "AUDIO" are often recommended.

Wait, but maybe I should go with a more common starting word. Let me recall, some people use "ADIEU" or "SOARE" because they contain many vowels and useful consonants. Alternatively, "STARE" is another option. Hmm. Let me pick a word that has a good spread of letters. How about "CRANE"? Let me think. C, R, A, N, E. Contains two vowels (A and E), and some common consonants. That might be a good start.

Alternatively, if I choose "STARE," that's S, T, A, R, E. Those are all high-frequency letters. Let's decide. Maybe "CRANE" is better because it includes C and N, which are a bit less common but still useful